### Imports

In [16]:
import os
import pandas as pd
import numpy as np

### Parameters

In [17]:
num_nodes = 100000  # quantidade de nós
num_edges = 200000  # quantidade de arestas

### Dataset

In [ ]:
def generate_realistic_edge(is_good_link):
    if is_good_link:
        rssi_mean = -55
        rssi_std = 8
        rssi = np.random.normal(rssi_mean, rssi_std)
        rssi = np.clip(rssi, -85, -30)
        
        etx = 1 + np.random.exponential(0.8)
        etx = np.clip(etx, 1, 8)
        
        delay = np.random.lognormal(2.3, 0.6)
        delay = np.clip(delay, 1, 80)
        
        busy = np.random.beta(2, 5)
        busy = np.clip(busy, 0.05, 0.7)
        
    else:
        rssi_mean = -75
        rssi_std = 10
        rssi = np.random.normal(rssi_mean, rssi_std)
        rssi = np.clip(rssi, -95, -45)
        
        etx = 1 + np.random.gamma(3, 1.2)
        etx = np.clip(etx, 1.5, 15)
        
        delay = np.random.lognormal(3.5, 0.8)
        delay = np.clip(delay, 10, 150)
        
        busy = np.random.beta(5, 2)
        busy = np.clip(busy, 0.1, 1.0)
    
    rssi += np.random.uniform(-0.05, 0.05)
    etx += np.random.uniform(-0.01, 0.01)
    delay += np.random.uniform(-0.2, 0.2)
    busy += np.random.uniform(-0.005, 0.005)
    
    rssi = np.clip(rssi, -95, -30)
    etx = np.clip(etx, 1.0, 15)
    delay = np.clip(delay, 1, 150)
    busy = np.clip(busy, 0.0, 1.0)
    
    if rssi < -70:
        etx *= np.random.uniform(1.1, 1.4)
        delay *= np.random.uniform(1.05, 1.25)
        etx = min(etx, 15)
    
    if busy > 0.6:
        delay *= np.random.uniform(1.2, 1.6)
        etx *= np.random.uniform(1.05, 1.2)
        delay = min(delay, 150)
        etx = min(etx, 15)
    
    if etx > 7:
        delay *= np.random.uniform(1.1, 1.3)
        delay = min(delay, 150)
    
    rssi = round(rssi, 2)
    etx = round(etx, 3)
    delay = round(delay, 2)
    busy = round(busy, 4)
    
    return rssi, etx, delay, busy


edges = []
label_0_count = 0
label_1_count = 0
target_per_label = num_edges // 2

existing_edges = set()

while len(edges) < num_edges:
    src = np.random.randint(0, num_nodes)
    dest = np.random.randint(0, num_nodes)
    
    if src == dest or (src, dest) in existing_edges or (dest, src) in existing_edges:
        continue
    
    existing_edges.add((src, dest))
    
    if label_0_count >= target_per_label:
        target_label = 1
    elif label_1_count >= target_per_label:
        target_label = 0
    else:
        target_label = 1 if np.random.random() < 0.52 else 0
    
    is_good = (target_label == 1)
    
    rssi, etx, delay, busy = generate_realistic_edge(is_good)
    
    label = target_label
    
    if label == 0:
        label_0_count += 1
    else:
        label_1_count += 1
    
    edges.append([src, dest, rssi, etx, delay, busy, label])

### Save

In [19]:
df = pd.DataFrame(edges, columns=["src", "dest", "rssi", "etx", "delay", "busy_fraction", "label"])
os.makedirs("../datasets", exist_ok=True)
df.to_csv("../datasets/edges_dataset.csv", index=False)

print("Dataset salvo em ../datasets/edges_dataset.csv")

Dataset salvo em ../datasets/edges_dataset.csv
